<a href="https://colab.research.google.com/github/usshaa/GoogleColab/blob/main/Langchain_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install langchain langchain-community huggingface_hub transformers faiss-cpu sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.1 MB/s eta 0:00:00


In [25]:
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

In [26]:
# prompt: use python dotenv module to load api key

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("HF_TOKEN")

In [27]:
# Initialize LLM with a smaller model
llm = HuggingFaceHub(repo_id="google/flan-t5-large", huggingfacehub_api_token=api_key)

In [28]:
# Example using the LLM
prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}"
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is the capital of France?"
response = llm_chain.run(question)
print(response)

paris


In [30]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [31]:
# Example using embeddings and a vector database (requires text)
text = "This is a sample document. It contains some text about a topic python."
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.create_documents([text])
db = FAISS.from_documents(docs, embeddings)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
query = "What is this document about?"
result = qa.run(query)
result


'Python'

In [32]:
# Example 3: Summarization
from langchain.chains.summarize import load_summarize_chain

In [33]:
# Load a summarization chain
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

In [34]:
# Example text to summarize
text_to_summarize = """
The sun shone brightly on the bustling city streets, a stark contrast to the quiet hum of the nearby park.
Birds chirped merrily in the trees, their melodies intertwining with the distant rumble of traffic.
People strolled along the pathways, some deep in conversation, others lost in their own thoughts.
Children chased pigeons across the grassy lawns, their laughter echoing through the air.
The gentle breeze rustled the leaves, creating a soothing symphony that accompanied the urban symphony.
"""

In [36]:
# Convert the text string into a Document object
from langchain.docstore.document import Document
docs_to_summarize = [Document(page_content=text_to_summarize)]

In [37]:
# Summarize the text
summary = summarize_chain.run(docs_to_summarize) # Pass the list of Document object
print(f"Summary:\n{summary}")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Summary:
The park is a symphony of life and music.


In [38]:
# Example 4: Question Answering with a different chain type
qa_map_reduce = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=db.as_retriever())
query_map_reduce = "What is this document about?"
result_map_reduce = qa_map_reduce.run(query_map_reduce)
print(f"Map Reduce Result:\n{result_map_reduce}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1441 > 1024). Running this sequence through the model will result in indexing errors


Map Reduce Result:
Python


In [39]:
# Example 5:  Using a different prompt template
new_prompt = PromptTemplate(
    input_variables=["question"],
    template="Provide a concise answer to the following question: {question}"
)
new_llm_chain = LLMChain(prompt=new_prompt, llm=llm)
new_question = "What is the meaning of life?"
new_response = new_llm_chain.run(new_question)
print(f"New Response:\n{new_response}")


New Response:
Life is the process of acquiring knowledge and becoming an informed citizen.
